# Lezione 9

## Parsing top-down direzionale (caso generale)

In [ ]:
from liblet import (
  Production, 
  Grammar, 
  Derivation, 
  TopDownInstantaneousDescription,
  Queue, 
  Stack, 
  animate_derivation,
  ProductionGraph, 
  side_by_side
)


## Imitando le derivazioni leftmost…

In [ ]:
# fig. 6.1, pag. 165

G = Grammar.from_string("""
S -> a B | b A
A -> a | a S | b A A
B -> b | b S | a B B
""")
G

Grammar(N={A, B, S}, T={a, b}, P=(S -> a B, S -> b A, A -> a, A -> a S, A -> b A A, B -> b, B -> b S, B -> a B B), S=S)

In [ ]:
# parola aabb

animate_derivation(Derivation(G).leftmost((0, 7, 5, 5)))

interactive(children=(IntSlider(value=0, description='n', max=4), Output(layout=Layout(height='300px'))), _dom…

In [ ]:
# non presente nel libro (caso che mostra meglio l'esigenza di una pila)

G = Grammar.from_string("""
S -> a B C
B -> a B | b
C -> a
""")

In [ ]:
# parola aaba

animate_derivation(Derivation(G).leftmost((0, 1, 2, 3)))

interactive(children=(IntSlider(value=0, description='n', max=4), Output(layout=Layout(height='300px'))), _dom…

# Simulare la computazione del NPDA 

La simulazione si basa su visite del DAG delle computazioni. In ogni nodo conserviamo la descrizione istantanea (a cui abbiamo aggiunto la derivazione che ha condotto a tale derivazione).

In [ ]:
i = TopDownInstantaneousDescription(G, 'aaba')
i

(), S♯, aaba♯

In [ ]:
i = i.predict(G.P[0])
i

(S -> a B C,), aBC♯, aaba♯

In [ ]:
i = i.match()
i

(S -> a B C,), BC♯, aaba♯

In [ ]:
i = i.predict(G.P[1])
i

(S -> a B C, B -> a B), aBC♯, aaba♯

In [ ]:
i = i.match()
i

(S -> a B C, B -> a B), BC♯, aaba♯

In [ ]:
i = i.predict(G.P[2])
i

(S -> a B C, B -> a B, B -> b), bC♯, aaba♯

In [ ]:
i = i.match()
i

(S -> a B C, B -> a B, B -> b), C♯, aaba♯

In [ ]:
i = i.predict(G.P[3])
i

(S -> a B C, B -> a B, B -> b, C -> a), a♯, aaba♯

In [ ]:
i = i.match()
i

(S -> a B C, B -> a B, B -> b, C -> a), ♯, aaba♯

In [ ]:
i.is_done()

True

## La funzione "stato prossimo"

In [ ]:
def next_instdescrs(instdescr):
  if instdescr.is_done(): return []
  G = instdescr.G
  top = instdescr.top()
  if top in G.T:
    return [instdescr.match()] if top == instdescr.head() else []
  else:
    return [instdescr.predict(P) for P in filter(Production.such_that(lhs = top), G.P)]

# tengo da parte un riferimento a questa implementazione per dopo…
original_next_instdescrs = next_instdescrs

## Usando una visita in ampiezza

Visitiamo il DAG con una *coda*, teniamo da parte *tutte le derivazioni* che man mano troviamo. 

La visita "verbosa" stampa il contenuto dell'intera coda ad ogni iterazione.

Il parametro `first_only` fa in modo che la simulazione si arresti quando viene trovata la prima derivazione.

In [ ]:
def breadth_first(G, word, verbose = False, first_only = False):
  q = Queue()
  q.enqueue(TopDownInstantaneousDescription(G, word))
  derivations = []
  while q:
    if verbose:
      for i in q: print(i)
      print('-' * 60)
    curr = q.dequeue()
    if curr.is_done():
      derivations.append(curr.steps)
      if first_only: return derivations
    else:
      for nxt in next_instdescrs(curr): q.enqueue(nxt)
  return derivations

In [ ]:
# testiamolo sull'ultima grammatica vista

steps = breadth_first(G, 'aaba')
steps

[(S -> a B C, B -> a B, B -> b, C -> a)]

In [ ]:
Derivation(G).leftmost(steps[0])

S -> a B C -> a a B C -> a a b C -> a a b a

## Non solo GNF

A ben guardare, la simulazione "funziona" non solo per le GNF… se il lato destro mescola simboli terminali e non, finiranno nella pila del PDA e verranno gestiti (apparentemente) senza particolari problemi…

In [ ]:
# un altro test, fedele al libro di testo

# fig. 6.6, pag. 171

G = Grammar.from_string("""
S -> A B | D C 
A -> a | a A
B -> b c | b B c 
D -> a b | a D b 
C -> c | c C
""")
G

Grammar(N={A, B, C, D, S}, T={a, b, c}, P=(S -> A B, S -> D C, A -> a, A -> a A, B -> b c, B -> b B c, D -> a b, D -> a D b, C -> c, C -> c C), S=S)

In [ ]:
breadth_first(G, 'aabc', verbose = True)

(), S♯, aabc♯
------------------------------------------------------------
(S -> A B,), AB♯, aabc♯
(S -> D C,), DC♯, aabc♯
------------------------------------------------------------
(S -> D C,), DC♯, aabc♯
(S -> A B, A -> a), aB♯, aabc♯
(S -> A B, A -> a A), aAB♯, aabc♯
------------------------------------------------------------
(S -> A B, A -> a), aB♯, aabc♯
(S -> A B, A -> a A), aAB♯, aabc♯
(S -> D C, D -> a b), abC♯, aabc♯
(S -> D C, D -> a D b), aDbC♯, aabc♯
------------------------------------------------------------
(S -> A B, A -> a A), aAB♯, aabc♯
(S -> D C, D -> a b), abC♯, aabc♯
(S -> D C, D -> a D b), aDbC♯, aabc♯
(S -> A B, A -> a), B♯, aabc♯
------------------------------------------------------------
(S -> D C, D -> a b), abC♯, aabc♯
(S -> D C, D -> a D b), aDbC♯, aabc♯
(S -> A B, A -> a), B♯, aabc♯
(S -> A B, A -> a A), AB♯, aabc♯
------------------------------------------------------------
(S -> D C, D -> a D b), aDbC♯, aabc♯
(S -> A B, A -> a), B♯, aabc♯
(S -> A B, 

[(S -> A B, A -> a A, A -> a, B -> b c)]

## Usando una visita in profondità

Visitiamo il DAG con una *pila*, come prima teniamo da parte *tutte le derivazioni* che man mano troviamo. 

La visita "verbosa" stampa il contenuto dell'intera coda ad ogni iterazione.

Il parametro `max_steps` limita (se diverso da -1) il numero massimo di passi di simulazione.

In [ ]:
def depth_first(G, word, verbose = False, max_steps = -1):
  s = Stack()
  s.push(TopDownInstantaneousDescription(G, word))
  derivations = []
  steps = 0
  while s:
    if steps > max_steps > -1: break
    steps += 1
    if verbose:
      for i in s: print(i)
      print('-' * 60)
    curr = s.pop()
    if curr.is_done():
      derivations.append(curr.steps)
    else:
      for nxt in next_instdescrs(curr): s.push(nxt)
  return derivations

In [ ]:
# Ancora l'esempio con la grammatica di fig. 6.6, pag. 171

depth_first(G, 'aabc', verbose = True)

(), S♯, aabc♯
------------------------------------------------------------
(S -> A B,), AB♯, aabc♯
(S -> D C,), DC♯, aabc♯
------------------------------------------------------------
(S -> A B,), AB♯, aabc♯
(S -> D C, D -> a b), abC♯, aabc♯
(S -> D C, D -> a D b), aDbC♯, aabc♯
------------------------------------------------------------
(S -> A B,), AB♯, aabc♯
(S -> D C, D -> a b), abC♯, aabc♯
(S -> D C, D -> a D b), DbC♯, aabc♯
------------------------------------------------------------
(S -> A B,), AB♯, aabc♯
(S -> D C, D -> a b), abC♯, aabc♯
(S -> D C, D -> a D b, D -> a b), abbC♯, aabc♯
(S -> D C, D -> a D b, D -> a D b), aDbbC♯, aabc♯
------------------------------------------------------------
(S -> A B,), AB♯, aabc♯
(S -> D C, D -> a b), abC♯, aabc♯
(S -> D C, D -> a D b, D -> a b), abbC♯, aabc♯
(S -> D C, D -> a D b, D -> a D b), DbbC♯, aabc♯
------------------------------------------------------------
(S -> A B,), AB♯, aabc♯
(S -> D C, D -> a b), abC♯, aabc♯
(S -> D C, D -> 

[(S -> A B, A -> a A, A -> a, B -> b c)]

## E la ricorsione?

Che succede se la grammatica contiene produzioni ricorsive a sinistra, ossia della forma $A\to A\gamma$ (con $A$ raggiungibile)? 

Per prima cosa, osserviamo che il DAG delle computazioni diventa *infinito* in quanto deve contenere, per ogni $k>0$, tutte le derivazioni della forma $S\stackrel{*}{\to} \alpha A \beta \stackrel{k}{\to} \alpha A \gamma^k \beta$.

In [ ]:
# pag. 173

G = Grammar.from_string('S -> a | S b')
G

Grammar(N={S}, T={a, b}, P=(S -> a, S -> S b), S=S)

Se ci contentiamo della prima derivazione, sicuramente la visita in ampiezza la troverà

In [ ]:
breadth_first(G, 'ab', first_only = True)

[(S -> S b, S -> a)]

Diverso è il discorso delle visita in profondità: se sulla pila c'è $S$ e la prossima mossa è data da $S -> Sb$ l'altezza della pila cresce senza mai consumare alcun carattere dell'input. 

Per questa ragione, la simulazione non potrà scoprire la derivazione in nessun numero finito di passi!

In [ ]:
depth_first(G, 'ab', verbose = False, max_steps = 10) # non cambia mettendo numeri maggiori!

[]

### <span style="color: red">Rfilessione per casa</span>

Cambia qualcosa in presenza di altre forme di produzioni ricorsive, ossia della forma $A -> \alpha A \beta$? È importante sapere se $\alpha \stackrel{*}{\to} \varepsilon$ o meno? 

Per rispondere, pensate a cosa accade con le GNF (con cui la simulazione funziona sempre), in particolare, ci possono essere produzioni ricorsive nelle GNF? E ricorsive a sinistra?

## Un "trucco"

Se la grammatica non contiene ε-produzioni, si può osservare che se in una configurazione istantanea il numero di simboli sulla pila eccede il numero di simboli della parola che restano da elaborare, quella configurazione non potrà mai condurre ad una derivazione della parola (perché non ci sono abbastanza terminali da associare ai non terminali in pila).

Possiamo modificare la funzione `make_next_instdescrs` in modo tale da "potare" gli stati prossimi in cui la pila è più grande del resto dell'input.


In [ ]:
def next_instdescrs(curr):
  def productive(instdescr):
    return len(instdescr.stack) <= (len(instdescr.tape) - instdescr.head_pos)
  return list(filter(productive, original_next_instdescrs(curr)))

Con quetsa modifica funzionano entrambe le visite (senza ulteriori limiti sul numero di derivazioni trovate, o passi effettuati)!

In [ ]:
breadth_first(G, 'ab')

[(S -> S b, S -> a)]

In [ ]:
depth_first(G, 'ab', verbose = True)

(), S♯, ab♯
------------------------------------------------------------
(S -> a,), a♯, ab♯
(S -> S b,), Sb♯, ab♯
------------------------------------------------------------
(S -> a,), a♯, ab♯
(S -> S b, S -> a), ab♯, ab♯
------------------------------------------------------------
(S -> a,), a♯, ab♯
(S -> S b, S -> a), b♯, ab♯
------------------------------------------------------------
(S -> a,), a♯, ab♯
(S -> S b, S -> a), ♯, ab♯
------------------------------------------------------------
(S -> a,), a♯, ab♯
------------------------------------------------------------
(S -> a,), ♯, ab♯
------------------------------------------------------------


[(S -> S b, S -> a)]

### <span style="color: red">Riflessione per casa</span>

Perché questa soluzione è meglio di imporre limiti come `first_only` e `max_steps`?

## Persino l'ambiguità

Nel caso di una grammatica ambiguta (e con produzioni ricorsive), la simulazione con la "potatura" produce tutte le derivazioni…

In [ ]:
# la solita grammatica delle operazioni aritmetiche (ambigua rispetto alla precedenza)

G = Grammar.from_string('E -> E * E | E + E | e')
G

Grammar(N={E}, T={*, +, e}, P=(E -> E * E, E -> E + E, E -> e), S=E)

In [ ]:
breadth_first(G, 'e*e+e')

[(E -> E * E, E -> e, E -> E + E, E -> e, E -> e),
 (E -> E + E, E -> E * E, E -> e, E -> e, E -> e)]

In [ ]:
d0, d1 = depth_first(G, 'e*e+e')

In [ ]:
side_by_side(
  ProductionGraph(Derivation(G).leftmost(d0)),
  ProductionGraph(Derivation(G).leftmost(d1))
)

## <span style="color: red;">Riflessioni per casa</span>

Cosa succede se si consentono anche ε-produzioni? Il DAG delle computazioni è finito? Che modi ci possono essere di "potarlo"?